In [2]:
#!pip install keras-tuner

In [8]:
import keras
from keras import layers

def build_model(hp):
    units = hp.Int(name='units', min_value=16, max_value=64, step=16)
    model = keras.Sequential([
        layers.Dense(units, activation=keras.activations.relu),
        layers.Dense(10, activation=keras.activations.softmax)
    ])
    optimizer = hp.Choice(name='optimizer', values=['rmsprop', 'adam'])
    model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=[keras.metrics.SparseCategoricalAccuracy()]
    )
    return model

In [9]:
import keras_tuner as kt

class SimpleMLP(kt.HyperModel):
    def __init__(self, num_classes):
        self.num_classes = num_classes

    def build(self, hp):
        units = hp.Int(name='units', min_value=16, max_value=64, step=16)
        model = keras.Sequential([
            layers.Dense(units, activation=keras.activations.relu),
            layers.Dense(self.num_classes, activation=keras.activations.softmax)
        ])
        optimizer = hp.Choice(name='optimizer', values=['rmsprop', 'adam'])
        model.compile(
            optimizer=optimizer,
            loss=keras.losses.SparseCategoricalCrossentropy(),
            metrics=[keras.metrics.SparseCategoricalAccuracy()])
        return model

hypermodel = SimpleMLP(num_classes=10)

In [22]:
tuner = kt.BayesianOptimization(
    build_model,
    objective='val_sparse_categorical_accuracy',
    max_trials=100,
    executions_per_trial=2,
    directory='mnist_kt_test',
    overwrite=True)

tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}


In [23]:
import numpy as np

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape((-1, 28 * 28)).astype(np.float32) / 255
x_test = x_test.reshape((-1, 28 * 28)).astype(np.float32) / 255
x_train_full = x_train[:]
y_train_full = y_train[:]

In [24]:
num_val_samples = 10000
x_train, x_val = x_train[:-num_val_samples], x_train[-num_val_samples:]
y_train, y_val = y_train[:-num_val_samples], y_train[-num_val_samples:]
callback_list = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
]

tuner.search(
    x_train, y_train,
    batch_size=128,
    epochs=100,
    validation_data=(x_val, y_val),
    callbacks=callback_list,
    verbose=2)

Trial 100 Complete [00h 00m 19s]
val_sparse_categorical_accuracy: 0.974700003862381

Best val_sparse_categorical_accuracy So Far: 0.9759000241756439
Total elapsed time: 00h 27m 48s


In [18]:
a = np.zeros((2, 2))
b = np.ones((3, 3))
c = a
c[0, 0] = 2
d[:] = b[:]
d[0, 0] = 3
(a, b, c, d)

(array([[2., 0.],
        [0., 0.]]),
 array([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]]),
 array([[2., 0.],
        [0., 0.]]),
 array([[3., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]]))

In [25]:
top_n = 4
best_hps = tuner.get_best_hyperparameters(top_n)
best_hps

In [27]:
def get_best_epoch(hp):
    model = build_model(hp)
    callback_list = [
        keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10)
    ]
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=100,
        batch_size=128,
        callbacks=callback_list)
    val_loss_per_epoch = history.history['val_loss']
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
    print(f'best_epoch: {best_epoch}')
    return best_epoch

In [28]:
def get_best_trained_model(hp):
    best_epoch = get_best_epoch(hp)
    model = build_model(hp)
    model.fit(x_train_full, y_train_full, batch_size=128, epochs=int(best_epoch * 1.2))
    return model

In [29]:
best_models = []
for hp in best_hps:
    model = get_best_trained_model(hp)
    model.evaluate(x_test, y_test)
    best_models.append(model)

Epoch 1/100
391/391 [==============================] - 1s 1ms/step - loss: 0.4596 - sparse_categorical_accuracy: 0.8752 - val_loss: 0.2450 - val_sparse_categorical_accuracy: 0.9327
Epoch 2/100
391/391 [==============================] - 0s 1ms/step - loss: 0.2299 - sparse_categorical_accuracy: 0.9351 - val_loss: 0.1863 - val_sparse_categorical_accuracy: 0.9487
Epoch 3/100
391/391 [==============================] - 0s 821us/step - loss: 0.1785 - sparse_categorical_accuracy: 0.9488 - val_loss: 0.1585 - val_sparse_categorical_accuracy: 0.9563
Epoch 4/100
391/391 [==============================] - 0s 859us/step - loss: 0.1467 - sparse_categorical_accuracy: 0.9576 - val_loss: 0.1433 - val_sparse_categorical_accuracy: 0.9585
Epoch 5/100
391/391 [==============================] - 0s 875us/step - loss: 0.1236 - sparse_categorical_accuracy: 0.9641 - val_loss: 0.1290 - val_sparse_categorical_accuracy: 0.9622
Epoch 6/100
391/391 [==============================] - 0s 887us/step - loss: 0.1065 - spa

In [30]:
x_train.shape, x_train_full.shape

((50000, 784), (60000, 784))